# Tactics

>This document is © 2015-2019 [tom-r](https://github.com/tom-r) and reproduced here from the [original source](https://github.com/tom-r/tactics_pi/blob/e97aede98dc203e6ec1f1ce17c6ff19947fe1b09/tactics_pi.odt), together with the source code containing tactic_pi algorithms and methods under the GNU General Public License v3 (GPL v3) – Likewise, modifications in [DashT software](https://github.com/canne/dashboard_tactics_pi) into those algorithms and their description in this document are © 2020 [canne](https://github.com/canne/dashboard_tactics_pi), and published  under the same license. Please feel free to fork and copy this work  but if you plan to publish it, please keep this notice unmodified in the document together with any copyright statement in the source code.

## Introduction

Everything started with the question : _“Do we make it around the edge of that island when we tack now and sail the same apparent wind angle on the other tack?“_

>**Disclaimer**: This is still alpha code (not even beta), and you should not use it for live – real sailing. I will not be liable for any harm, damage or whatever strange things happen if you use this plugin and rely on its data.

## What Tactics can do?

- Calculate true wind data : TWA, TWD, TWS from true heading (HDT), speed through water (STW) and app. Wind speed (AWS), with optional correction by heel-angle. Calculation is enabled via a preference setting and disables available true wind data from the bus throughout the tactics_pi plugin ;

- Calculate the “leeway”, the boat drift based on heel. A common formula is used for that purpose ;

- Calculate the surface sea current and display it as single instruments (current speed/direction) as part of the “Bearing compass” or as overlay on the chart (semi transparent). The routines take boat heel and leeway into account. If you don't have a heel sensor, there is a simply workaround, see below. Current display on the chart can be disabled by a preference setting ;

- Calculate and display the boat laylines for the current tack, and the same TWA on the other tack. Sea current is taken into account, if available ! Laylines may be toggled on/off.  Adjustable length and max. width (triangle, with one corner at the boat) of the boat laylines. The layline width reflects the boat's yawing (COG changes over time) ;

- You can load a polar file and calculate/display performance data, like actual VMG (velocity made good up-/downwind), Target-VMG, Target-TWA  (the opt. TWA up-/downwind), CMG (course made good towards a waypoint), Target-CMG (opt. CMG angle and speed), polar speed (the speed you should be able to sail at current TWA/TWS based on your polar) ;

- It can display the actual polar as overlay on the chart, including markers for Target-VMG/CMG angles. Just adopt your course and place your heading pointer on one of the markers, and you sail Target-VMG/CMG based on your polar ;

- You can set a (one !) temporary tactics waypoint and display the laylines to the mark, based on a Target-TWA calculation, while taking your polar into account ;

- It has a “dial instruments” called “Bearing compass”. Boat true heading (HDT)  points “up”, it shows the boat laylines as well, the surface current, a pointer to the waypoint (either set manually as the temporary Tactics waypoint or read from a NMEA RMB sentence), needles for AWA and TWA, markers for the Target-VMG/CMG angles ;

- It has a “Polar compass” instrument, graphically displaying the actual polar ring and markers for Bearing to WP, Target-VMG angles and Target-CMG angles ;

- It has an “Average Wind” instrument with an adjustable averaging time, which displays graphically the average wind and its deviations to port / starboard

- It can create specific NMEA performance records with the purpose to export them to the displays of your instruments. You can now e.g. calculate the polar target speed in the plugin and send it to your instrument display outside in the cockpit. Currently only available for NKE, but may be enhanced in the future ;

- It can export the data from BaroHistory, WindHistory and PolarPerfomance instruments to a flat text file with selectable export rate and column separator (via ini file). The export will either create a new, or append to an existing export file ;

- Various Tactics settings are grouped in a separate tab in the Preferences configuration dialog of _DashT_. To access the preferences dialog, right mouse click on any _DashT_ instrument, then select “Preferences" (**8**);

<img src="img/s_001_DashT_prefs_context_menu.png"
alt="Context menu" width="200">[(zoom)](img/001_DashT_prefs_context_menu.png)

- You'll find all seetings in a separate tab _“Tactics Performance Parameters”_ (**1**):

<img src="img/s_005_DashT_Tactics_Perf_Parameters_dlg.png"
alt="Tactics Performance Parameter Access" width="200">[(zoom)](img/005_DashT_Tactics_Perf_Parameters_dlg.png)

- There are pop-up _ToolTips_ available for almost all of the Tactics preferences. Just hover the mouse over any of the settings to get explanation and usage description.

<img src="img/s_005_DashT_Tactics_Parameters_Tooltip.png"
alt="Tooltips" width="400">[(zoom)](img/005_DashT_Tactics_Parameters_Tooltip.png)

## Prerequisites

- You will need to activate OpenGL, if you want to use the chart overlay functions ;
- SOG, COG from the GPS ;
- True Heading from an electronic compass. If not available, magnetic heading will do, as long as you have magnetic variance available, either from the GPS RMC sentence or from the wmm_pi plug-in - note that the wmm_pi plug-in's value has the priority (Dashboard heritage) so you may want to disable wmm_pi plug-in if your own preference is the RMC sentence ;
- Boat speed through water **STW** from a log / “paddlewheel” sensor (keep it clean!) ;
- Apparent wind angle **AWA** and Apparent Wind Speed **AWS** ;
- Heel sensor which supplies your boat's heel angle, also calld "roll". If not available, there's a workaround with manual input in preferences ;
- You need a polar file of your boat to use all polar based performance calculations, explained in a separate section in this document ;
- Calibrated sensors or corection to AWA, Compass HDG/HDT, STW (Speed through water), and AWS (apparent wind speed), as good as possible. Especially the compass heading calibration tends to be neglected. However, this is vital for a  proper surface current calculation. All I can say is :   sh*** in – sh*** out …   ;-) .

The terms CMG and VMG used in this documentation are not unique across instrument manufacturers and other sailing software packages, even in OpenCPN and in Dashboard "VMG" is used in routing functions! This software will still use both terms, familiar to the intended target users, both in the software and here, in this documentation. However you can change both terms by setting 2 preferences in the ini/conf-file file. See the corresponding chapter explaing the parameters in that file.

See also the Terminology at the very end for explanation of terms

## Without Polar

### Calculate true wind data

The plug-in can calculate true wind data (TWA, TWS, TWD) but keeps the calculated data inside the plug-in. (it does not broadcast it to OpenCPN!)

True wind calculation is done silently if you don’t have TWA, TWS, TWD available in your NMEA stream.  Furthermore you can force the true wind calculation in the plug-in by a preference setting. If the tick _“Force True Wind Calculation”_ is set 

<img src="img/s_015_DashT_Tactics_tw_calcs_tickboxes.png"
alt="DashT Tactics TW Tickboxes" width="400">[(zoom)](img/015_DashT_Tactics_tw_calcs_tickboxes.png)

It does not matter if TWA, TWS and TWD are already available on the system or not. Calculation is done in the plug-in then.  It does calculate TWA, TWS and TWD.

This is useful in case, for example if you have a heel sensor but which is not integrated in your instrument bus. You can use the corrections to get more accurate true wind data by taking into account the heel (roll).

Input is AWA, AWS, STW, and for TWD also true heading HDT.

If you don't have HDT on your system bus (but only HDG), you can use wmm_pi plug-in: it supplies the magnetic variation and if it is enabled, it value is fed into _DashT_ by OpenCPN and its value is taken into account to calculate HDT from HDG.

If you have a heel sensor in your system, and its data is available to _DashT_, you can use the three following corrections:

1. **Correct STW with Leeway** : the plug-in can calculate your leeway (drift angle) based on on your heel sensor (see below). That means your boat is possibly moving sideways, which adds an error to the True Wind calculation. Standard instruments normally do not take this effect into account, as far as I know. NKE does this correction in its regatta processor only, but not on their normal instruments.
1. **Correct AWS/AWA with Heel** : This option corrects your AWS and AWA data by the heel angle. Use this option with great care !. Manufacturers normally already do correct this, if you have a heel sensor integrated in your instrument bus. O will simply receive the already corrected data for AWS / AWA then. The result would be wrong data ! I implemented this option for those sailors using an external (or DIY) heel sensor, which is not recognized by their instrument system. You'll get a warning popup as soon as you set the tick.
1. **Use SOG instead of STW for True Wind Calc** : Replaces STW (Speed through water, the “log”) with SOG (from the GPS) in the internal true wind calculation.  The idea is simply to have a fallback for the true wind calculation in case your log fails and also to eliminate side effects on the calculation by surface current. Some like the option to refer to the ground and some prefer to refer to the water - make your choice!

>**NOTE**: if you have issues with the true wind calculation in _DashT_, take a look at the OpenCPN log-file. _DashT_ collects the information needed by the Tactics algorithm and if something is missing, it is reported in the log file to help the troubleshooting.

### Calculate Leeway

Leeway describes the drift of the boat due to the force of the wind. Leeway is the basic input for the surface current calulation described later on. Input for the leeway calculation is your heel angle. Normally you'd say : the more you heel, the more you drift. But that's only part of the truth. Other significant inputs are boat speed and the shape of your hull.

A widely (NKE, B&G,...) used formula calculates the leeway with 3 input values : heel, boat speed (STW), and hullshape-factor.

The attribute in the preferences is called _“Boat's Leeway factor [0..20]:”_

The input range is 0...20, 10 is a good value to start with.

f you don't have a heel sensor on board, you can either set a fixed value ( e.g. 0 when motoring without sails), or try to set up a very simple _“heel polar”_:

<img src="img/s_020_DashT_Tactics_heel_polar_preferences.png"
alt="Heel Polar" width="200">[(zoom)](img/020_DashT_Tactics_heel_polar_preferences.png)

The idea of the _“heel polar”_ is that almost every magnetic compass installed in the cockpits has a scale, where you can read the degrees of heel: simply compare the _DashT_ Dashboard instruments display, for TWA and TWS, read the values from the scale and put it into the small table above.

>**NOTE**: Please make sure to read the True Wind Angle (TWA) and the True Wind Speed (TWS) and not apparent wind angle and speed (AWA/AWS).

I tried it on my own boat, comparing the heel polar values with those of my sensor. It works astonishingly well.

Even if you use the heel-polar, you have to estimate the _“Boat's Leeway factor [0..20]:”_

You have 3 choices for heel input, depending on where you set the radio button in the preferences. You can switch the radio buttons forth and back while sailing to compare the results, or as a tactician to take into account different sailing conditions.

<img src="img/s_025_DashT_Tactics_Leeway_factor_preferences.png"
alt="Leeway factor" width="300">[(zoom)](img/025_DashT_Tactics_Leeway_factor_preferences.png)

The attribute _“fixed/max Leeway [°]:”_ has a dual purpose:

1. The given value is always taken into account as maximum possible Leeway value. In the screenshot below, I set it to 30°. If your heel polar or calculation with the formula above outputs values >30°, the program takes 30° ;
1. If you set the radio button here, the routines always take 30°, no matter what your sensor calculates or your heel-polar would tell you.

### Calculate the surface current

If you compare your HDT and COG vectors in OpenCPN (the 2 forward vectors on the chart at your boat), the difference between both is a mixture between Leeway (the boat's drift) and surface current. Once we can determine Leeway, the rest is surface current.

The surface current calculation is simply a triangle calculation with vectors.

Always seen from the current position, the first vector is HDT (degrees) / STW (length).

As your boat drifts with the wind, the second vector is “course through water” (CRS, degrees) and STW (length).

Course through water is actually HDT with applied leeway.

The resulting vector between CRS/STW and COG/SOG is the surface current.

<img src="img/s_030_DashT_Tactics_current_vector_calculation.png"
alt="Current vector calculation" width="240">[(zoom)](img/030_DashT_Tactics_current_vector_calculation.png)

To calculate the current, you need as input HDT, STW, Leeway, COG and SOG and your GPS latitude / longitude.

In the preferences you can set the damping factor for the current calculations : the lower the values are the more filtering is applied, and the reading gets more stable. On the other hand, it starts lagging a bit.
The lower the value, the more damping is applied. You can do experimenting in the range of 0.001  to 0.025. Keep this value at the lower end, then start to increase, until it gets unstable.

<img src="img/s_035_DashT_Tactics_current_damping_factor_preferences.png"
alt="Current Damping Factor" width="280">[(zoom)](img/035_DashT_Tactics_current_damping_factor_preferences.png)

>**NOTE**: Tactics algorithms currently always corrects STW with Leeway for the current calculation, independent of the preference setting  _“Correct STW with leeway”_. This may change in a future release.

<img src="img/s_.png"
alt="XXXX" width="200">[(zoom)](img/.png)

<img src="img/s_.png"
alt="XXXX" width="200">[(zoom)](img/.png)

<img src="img/s_.png"
alt="XXXX" width="200">[(zoom)](img/.png)

<img src="img/s_.png"
alt="XXXX" width="200">[(zoom)](img/.png)

<img src="img/s_.png"
alt="XXXX" width="200">[(zoom)](img/.png)